<a href="https://colab.research.google.com/github/luquelab/bioinformatics-teamCanes/blob/main/notebook/main_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Sequence Analysis and Functional Prediction Pipeline Notebook

This notebook performs a thorough analysis of a group of nucleotide sequences. The pipeline includes:
- **Sequence Analysis**: Uploading FASTA sequences, computing sequence lengths, GC content, and performing translations.
- **Visualization**: Plotting sequence length and GC content, generating similarity matrices with heatmaps.
- **Alignments & Phylogenetic Trees**: Pairwise alignments, computing percent identities, and creating phylogenetic trees.
- **Functional & Organism Prediction**: Running online BLAST and HMMER searches to predict potential functions and organismal origins.
- **Directory Structure**: The code automatically creates a directory structure to keep data, results, and intermediate files organized.

This notebook is intended to be run in a Google Colab environment because it utilizes the `google.colab` module for file uploads and drive access.

To test this pipeline, use the "sequences.fasta" test file.


In [ ]:
!pip install biopython lxml

import os
import re
import shutil
import subprocess
import time
import requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from Bio import SeqIO, Phylo
from Bio.SeqRecord import SeqRecord
from Bio import pairwise2
from Bio.Phylo.TreeConstruction import DistanceMatrix, DistanceTreeConstructor
from Bio.Blast import NCBIWWW, NCBIXML
from lxml import etree
from google.colab import files
from google.colab import drive

## Directory Structure Setup

This section defines and creates a directory structure to organize your project files. The structure includes folders for data, results (with several sub-folders for different analyses), and binaries.

Modify your root_dir variable to match desired directory name.

In [ ]:
# Change the assignment root name as needed (e.g., project_x_analysis)
root_dir = "project_x_analysis"
dirs = {
    "data": os.path.join(root_dir, "data"),
    "bin": os.path.join(root_dir, "bin"),
    "sequence_properties": os.path.join(root_dir, "results", "sequence_properties"),
    "alignments": os.path.join(root_dir, "results", "alignments"),
    "phylogenetic_tree": os.path.join(root_dir, "results", "phylogenetic_tree"),
    "functional_prediction": os.path.join(root_dir, "results", "functional_prediction"),
    "organism_origin": os.path.join(root_dir, "results", "organism_origin"),
}

for d in dirs.values():
    os.makedirs(d, exist_ok=True)

# Define the assignment bin directory
bin_dir = os.path.join(root_dir, "bin")
os.makedirs(bin_dir, exist_ok=True)

## Part 1: Sequence Analysis Pipeline

This portion of the notebook performs several tasks:
1. **Sequence Upload**: Prompts the user to upload a FASTA file.
2. **Sequence Properties Calculation**: Reads the FASTA file, computes basic properties such as the sequence length (in bp and kb) and GC content, and translates the nucleotide sequences into protein sequences.
3. **Visualizations**: Plots sequence length and GC content bar charts and saves the figures.
4. **Pairwise Alignments and Similarity Calculations**: Uses pairwise global alignments to compute percent identity between sequences (for both nucleotide and translated protein sequences) and outputs the results.
5. **Similarity Matrices and Heatmaps**: Constructs full similarity matrices and visualizes them as heatmaps.
6. **Phylogenetic Tree Construction**: Generates distance matrices and constructs neighbor-joining (NJ) trees for both nucleotide and protein sequences.

### Step 1: Upload and Store Sequences File

Use the interactive file upload widget provided by Google Colab to upload your FASTA file. The file is then copied into the `data` folder.

An example .fasta is provided in the repo as "sequences.fna".

In [ ]:
print("Please upload your FASTA file (e.g., sequences.fna) ...")
uploaded = files.upload()  # Interactive file upload for FASTA file
filename = list(uploaded.keys())[0]
print(f"Uploaded file: {filename}")

# Move the uploaded file to the data/ folder.
data_file = os.path.join(dirs["data"], "sequences.fasta")
shutil.copy(filename, data_file)
print(f"Copied file to: {data_file}")

Please upload your FASTA file (e.g., sequences.fna) ...


Saving sequences.fasta to sequences (1).fasta
Uploaded file: sequences (1).fasta
Copied file to: project_x_analysis/data/sequences.fasta


### Step 2: Load Sequences and Compute Basic Properties

This cell:
- Loads the sequences from the uploaded FASTA file.
- Calculates the length (in bp and kb) and GC content for each sequence.
- Translates each nucleotide sequence into a protein sequence (translation halts at the first stop codon).
- Saves these computed properties into a CSV file.
- Generates bar charts for sequence lengths and GC content.

In [ ]:
records = list(SeqIO.parse(data_file, "fasta"))
print(f"You successfully loaded {len(records)} sequences.")

def compute_gc(seq):
    """Compute GC content percentage."""
    seq = seq.upper()
    return 100 * (seq.count("G") + seq.count("C")) / len(seq) if len(seq) > 0 else 0

data_props = []
protein_records = []
for record in records:
    seq_str = str(record.seq)
    length_bp = len(seq_str)
    gc_content = compute_gc(seq_str)
    # Translate (stop at the first stop codon)
    protein_seq = record.seq.translate(to_stop=True)
    protein_records.append(SeqRecord(protein_seq, id=record.id, description="Translated sequence"))
    data_props.append({
        "Sequence_ID": record.id,
        "Length_bp": length_bp,
        "Length_kb": length_bp / 1000,
        "GC_Content (%)": gc_content
    })

df_props = pd.DataFrame(data_props)
print(df_props)

# Save sequence properties.
props_csv = os.path.join(dirs["sequence_properties"], "properties.csv")
df_props.to_csv(props_csv, index=False)
print(f"Sequence properties saved to: {props_csv}")

You successfully loaded 5 sequences.
  Sequence_ID  Length_bp  Length_kb  GC_Content (%)
0  EF990648.1       1186      1.186       42.327150
1  GU220721.1       1194      1.194       48.576214
2  JX429970.1       1216      1.216       47.286184
3  KU291912.1       1310      1.310       48.778626
4  JN107639.1       1350      1.350       48.148148
Sequence properties saved to: project_x_analysis/results/sequence_properties/properties.csv


/usr/local/lib/python3.11/dist-packages/Bio/Seq.py:2879: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


#### Visualizing Sequence Properties

The following code generates bar charts for:
- **Sequence Lengths (in kb)**
- **GC Content (%)**

Each figure is saved as a high-resolution PNG in the `sequence_properties` folder.

In [ ]:
# Plot and save sequence lengths.
plt.figure(figsize=(8, 6))
plt.bar(df_props["Sequence_ID"], df_props["Length_kb"])
plt.xlabel("Sequence ID")
plt.ylabel("Length (kb)")
plt.title("Sequence Lengths")
plt.xticks(rotation=45)
plt.tight_layout()
lengths_png = os.path.join(dirs["sequence_properties"], "lengths.png")
plt.savefig(lengths_png, dpi=300)
plt.show()
print(f"Length plot saved to: {lengths_png}")

# Plot and save GC content.
plt.figure(figsize=(8, 6))
plt.bar(df_props["Sequence_ID"], df_props["GC_Content (%)"])
plt.xlabel("Sequence ID")
plt.ylabel("GC Content (%)")
plt.title("GC Content of Sequences")
plt.xticks(rotation=45)
plt.tight_layout()
gc_png = os.path.join(dirs["sequence_properties"], "gc_content.png")
plt.savefig(gc_png, dpi=300)
plt.show()
print(f"GC content plot saved to: {gc_png}")

# Save translated protein sequences to FASTA.
translated_faa = os.path.join(dirs["sequence_properties"], "sequences_translated.faa")
SeqIO.write(protein_records, translated_faa, "fasta")
print(f"Translated protein sequences saved to: {translated_faa}")

### Step 3: Pairwise Alignments & Similarity Calculations

This section calculates the percent identity between pairs of sequences:
- **Pairwise Alignments**: Global alignments for both nucleotide and protein sequences are computed.
- **Percent Identity**: Functions are provided to compute the percent identity from the alignments.
- The results are saved in a CSV file along with the formatted alignments.

In [ ]:
def calc_percent_identity(aln_seq1, aln_seq2):
    matches = sum(a == b for a, b in zip(aln_seq1, aln_seq2) if a != '-' and b != '-')
    alignment_length = max(len(aln_seq1), len(aln_seq2))
    return (matches / alignment_length) * 100

def compute_percent_identity(seq1, seq2):
    aln = pairwise2.align.globalxx(seq1, seq2, one_alignment_only=True)[0]
    return calc_percent_identity(aln.seqA, aln.seqB)

similarity_data = []
nuc_alignment_file = os.path.join(dirs["alignments"], "sequence_alignments_nucleotide.fna")
prot_alignment_file = os.path.join(dirs["alignments"], "sequence_alignments_proteins.faa")

with open(nuc_alignment_file, "w") as nuc_out, open(prot_alignment_file, "w") as prot_out:
    for i in range(len(records)):
        for j in range(i + 1, len(records)):
            rec1, rec2 = records[i], records[j]
            # Nucleotide alignment
            aln_nuc = pairwise2.align.globalxx(rec1.seq, rec2.seq, one_alignment_only=True)[0]
            nuc_out.write(f"Alignment between {rec1.id} and {rec2.id}:\n")
            nuc_out.write(pairwise2.format_alignment(*aln_nuc))
            nuc_out.write("\n" + "=" * 80 + "\n\n")
            nuc_id = calc_percent_identity(aln_nuc.seqA, aln_nuc.seqB)
            # Protein alignment
            prot_aln = pairwise2.align.globalxx(protein_records[i].seq, protein_records[j].seq, one_alignment_only=True)[0]
            prot_out.write(f"Alignment between {rec1.id} and {rec2.id}:\n")
            prot_out.write(pairwise2.format_alignment(*prot_aln))
            prot_out.write("\n" + "=" * 80 + "\n\n")
            prot_id = calc_percent_identity(prot_aln.seqA, prot_aln.seqB)
            similarity_data.append({
                "Sequence1": rec1.id,
                "Sequence2": rec2.id,
                "Nucleotide_percent_identity": round(nuc_id, 2),
                "Protein_percent_identity": round(prot_id, 2)
            })

similarity_csv = os.path.join(dirs["alignments"], "sequence_similarities.csv")
df_similarity = pd.DataFrame(similarity_data)
df_similarity.to_csv(similarity_csv, index=False)
print(f"Pairwise similarity CSV saved to: {similarity_csv}")

### Step 4: Full Similarity Matrices and Heatmaps

This section generates full similarity matrices for nucleotide and protein sequences by computing pairwise percent identities. The matrices are saved as CSV files and visualized as heatmaps.

In [ ]:
num_nuc = len(records)
nuc_sim_matrix = np.zeros((num_nuc, num_nuc))
for i in range(num_nuc):
    for j in range(num_nuc):
        nuc_sim_matrix[i, j] = compute_percent_identity(str(records[i].seq), str(records[j].seq))
df_nuc_sim = pd.DataFrame(nuc_sim_matrix, index=[r.id for r in records], columns=[r.id for r in records])
nuc_matrix_csv = os.path.join(dirs["alignments"], "nucleotide_similarity_matrix.csv")
df_nuc_sim.to_csv(nuc_matrix_csv, index=True)
print(f"Nucleotide similarity matrix saved to: {nuc_matrix_csv}")

num_prot = len(protein_records)
prot_sim_matrix = np.zeros((num_prot, num_prot))
for i in range(num_prot):
    for j in range(num_prot):
        prot_sim_matrix[i, j] = compute_percent_identity(str(protein_records[i].seq), str(protein_records[j].seq))
df_prot_sim = pd.DataFrame(prot_sim_matrix, index=[r.id for r in protein_records], columns=[r.id for r in protein_records])
prot_matrix_csv = os.path.join(dirs["alignments"], "protein_similarity_matrix.csv")
df_prot_sim.to_csv(prot_matrix_csv, index=True)
print(f"Protein similarity matrix saved to: {prot_matrix_csv}")

plt.figure(figsize=(8, 6))
plt.imshow(df_nuc_sim, interpolation='nearest', cmap='viridis')
plt.colorbar(label='Percent Identity (%)')
plt.xticks(range(num_nuc), df_nuc_sim.columns, rotation=45)
plt.yticks(range(num_nuc), df_nuc_sim.index)
plt.title("Nucleotide Similarity Matrix")
plt.tight_layout()
nuc_heatmap_png = os.path.join(dirs["alignments"], "nucleotide_similarity_heatmap.png")
plt.savefig(nuc_heatmap_png, dpi=300)
plt.show()
print(f"Nucleotide similarity heatmap saved to: {nuc_heatmap_png}")

plt.figure(figsize=(8, 6))
plt.imshow(df_prot_sim, interpolation='nearest', cmap='viridis')
plt.colorbar(label='Percent Identity (%)')
plt.xticks(range(num_prot), df_prot_sim.columns, rotation=45)
plt.yticks(range(num_prot), df_prot_sim.index)
plt.title("Protein Similarity Matrix")
plt.tight_layout()
prot_heatmap_png = os.path.join(dirs["alignments"], "protein_similarity_heatmap.png")
plt.savefig(prot_heatmap_png, dpi=300)
plt.show()
print(f"Protein similarity heatmap saved to: {prot_heatmap_png}")

### Step 5: Build Phylogenetic Trees

Using the computed similarity matrices, this section:
- Converts the similarity data into distance matrices.
- Constructs phylogenetic trees (neighbor-joining trees) for both nucleotide and protein sequences.
- Saves the trees in Newick format and also saves visualizations of the trees.

In [ ]:
def build_distance_matrix_with_diag(df_sim):
    names = list(df_sim.index)
    num_seqs = len(names)
    lower_triangle = []
    for i in range(num_seqs):
        row = []
        for j in range(i):
            distance = 1 - (float(df_sim.iat[i, j]) / 100.0)
            row.append(distance)
        row.append(0.0)
        lower_triangle.append(row)
    return DistanceMatrix(names, lower_triangle)

nuc_dm = build_distance_matrix_with_diag(df_nuc_sim)
constructor = DistanceTreeConstructor()
nuc_tree = constructor.nj(nuc_dm)
nuc_tree_file = os.path.join(dirs["phylogenetic_tree"], "tree_nucleotides.nwk")
Phylo.write(nuc_tree, nuc_tree_file, "newick")
print(f"Nucleotide tree (Newick) saved to: {nuc_tree_file}")
plt.figure(figsize=(10, 8))
Phylo.draw(nuc_tree, do_show=False)
nuc_tree_png = os.path.join(dirs["phylogenetic_tree"], "tree_nucleotides_visualization.png")
plt.savefig(nuc_tree_png, dpi=300, bbox_inches='tight')
plt.close()
print(f"Nucleotide tree visualization saved to: {nuc_tree_png}")

prot_dm = build_distance_matrix_with_diag(df_prot_sim)
prot_tree = constructor.nj(prot_dm)
prot_tree_file = os.path.join(dirs["phylogenetic_tree"], "tree_proteins.nwk")
Phylo.write(prot_tree, prot_tree_file, "newick")
print(f"Protein tree (Newick) saved to: {prot_tree_file}")
plt.figure(figsize=(10, 8))
Phylo.draw(prot_tree, do_show=False)
prot_tree_png = os.path.join(dirs["phylogenetic_tree"], "tree_proteins_visualization.png")
plt.savefig(prot_tree_png, dpi=300, bbox_inches='tight')
plt.close()
print(f"Protein tree visualization saved to: {prot_tree_png}")

print("\n=== Sequence Analysis Pipeline Completed ===")

## Part 2: Functional & Organism Prediction Pipeline

This section leverages online tools to predict the function of your sequences and their organismal origin. The workflow includes:
- **BLAST Search**: Each sequence is used to perform a BLAST search against the NCBI database. The resulting XML files are parsed to extract hit definitions and predicted organism information.
- **HMMER Search**: An online HMMER search (against the Pfam database) is conducted using the translated protein sequence to predict domain structures.
- The predictions are then aggregated and saved into CSV files.
- Finally, corresponding functional prediction files are copied to organism-specific folders for further analysis.

In [ ]:
blast_program = "blastx"
blast_db = "nr"
blast_hitlist_size = 5
use_online_hmmer = True   # Set to False if using local hmmscan
hmmdb_online = "pfam"

domain_predictions = []
organism_predictions = []
organism_overall = []

for record in records:
    seq_id = record.id
    # Create separate folders per sequence under functional_prediction and organism_origin
    seq_func_folder = os.path.join(dirs["functional_prediction"], seq_id)
    seq_org_folder  = os.path.join(dirs["organism_origin"], seq_id)
    os.makedirs(seq_func_folder, exist_ok=True)
    os.makedirs(seq_org_folder, exist_ok=True)

    # --- BLAST Search ---
    print(f"Running BLAST for {seq_id} ...")
    try:
        blast_handle = NCBIWWW.qblast(blast_program, blast_db, record.seq, hitlist_size=blast_hitlist_size)
        blast_xml = blast_handle.read()
        blast_handle.close()
    except Exception as e:
        print(f"Error during BLAST search for {seq_id}: {e}")
        blast_xml = ""
    blast_outfile = os.path.join(seq_func_folder, f"{seq_id}_blast.xml")
    with open(blast_outfile, "w") as f:
        f.write(blast_xml)
    print(f"BLAST results saved to {blast_outfile}")

    function_preds = []
    organism_preds = []
    try:
        with open(blast_outfile) as result_handle:
            blast_record = NCBIXML.read(result_handle)
            if blast_record.alignments:
                for hit in blast_record.alignments[:blast_hitlist_size]:
                    function_preds.append(hit.hit_def)
                    org_match = re.search(r'\[(.*?)\]', hit.hit_def)
                    if org_match:
                        organism_preds.append(org_match.group(1))
                    else:
                        organism_preds.append("Unknown")
            else:
                print(f"No BLAST hits found for {seq_id}")
    except Exception as e:
        print(f"Error parsing BLAST result for {seq_id}: {e}")

    function_pred = "; ".join(function_preds) if function_preds else "No hit"
    organism_pred = "; ".join(organism_preds) if organism_preds else "Unknown"
    organism_predictions.append([seq_id, organism_pred, function_pred])

    # --- HMMER Search ---
    protein_seq = record.seq.translate(to_stop=True)
    if use_online_hmmer:
        def run_online_hmmer(prot_seq, seq_id):
            submit_url = "https://www.ebi.ac.uk/Tools/hmmer/search/hmmscan"
            data = {"hmmdb": hmmdb_online, "seq": str(prot_seq), "domE": "1e-5"}
            headers = {"Expect": "", "Accept": "text/xml"}
            print(f"Submitting online HMMER job for {seq_id} ...")
            response = requests.post(submit_url, data=data, headers=headers)
            if response.status_code != 200:
                raise Exception(f"HMMER error for {seq_id}: {response.status_code}")
            return response.text
        try:
            hmm_xml = run_online_hmmer(protein_seq, seq_id)
            hmm_outfile = os.path.join(seq_func_folder, f"{seq_id}_hmm_online.xml")
            with open(hmm_outfile, "w") as hfile:
                hfile.write(hmm_xml)
            print(f"Online HMMER results saved to {hmm_outfile}")
        except Exception as e:
            print(f"Error during online HMMER for {seq_id}: {e}")
            hmm_xml = ""
    else:
        hmm_outfile = os.path.join(seq_func_folder, f"{seq_id}_hmm.tbl")
        cmd = f"hmmscan --tblout {hmm_outfile} Pfam-A.hmm {seq_id}_prot.faa"
        try:
            subprocess.run(cmd, shell=True, check=True)
        except Exception as e:
            print(f"Error running local hmmscan for {seq_id}: {e}")
            hmm_outfile = ""
        hmm_xml = ""

    def parse_hmmer_xml(xml_content):
        domains = []
        try:
            parser = etree.XMLParser(recover=True)
            root = etree.fromstring(xml_content.encode("utf-8"), parser=parser)
            data = root.find(".//data[@name='results']")
            if data is None:
                return domains
            for hit in data.findall("hits"):
                hit_name = hit.get("name") or "UnknownHit"
                for dom in hit.findall("domains"):
                    dom_name = dom.get("alihmmname") or hit_name
                    dom_evalue = dom.get("ievalue") or dom.get("evalue") or "N/A"
                    domains.append(f"{dom_name} (E={dom_evalue})")
        except Exception as e:
            print("Error parsing HMMER XML:", e)
        return domains

    domains = parse_hmmer_xml(hmm_xml) if hmm_xml else []
    domain_pred = "; ".join(domains) if domains else "No domains detected"
    domain_predictions.append([seq_id, domain_pred])
    organism_overall.append([seq_id, organism_pred])

    for f in os.listdir(seq_func_folder):
        shutil.copy(os.path.join(seq_func_folder, f), os.path.join(seq_org_folder, f))

df_domains = pd.DataFrame(domain_predictions, columns=["Sequence_ID", "Domain_Predictions"])
domain_csv = os.path.join(dirs["functional_prediction"], "domain_predictions.csv")
df_domains.to_csv(domain_csv, index=False)
print(f"Domain predictions saved to: {domain_csv}")

df_org_func = pd.DataFrame(organism_predictions, columns=["Sequence_ID", "Predicted_Organism", "Function_Prediction"])
org_func_csv = os.path.join(dirs["functional_prediction"], "organism_predictions.csv")
df_org_func.to_csv(org_func_csv, index=False)
print(f"Organism predictions saved to: {org_func_csv}")

df_org_overall = pd.DataFrame(organism_overall, columns=["Sequence_ID", "Predicted_Organism"])
org_overall_csv = os.path.join(dirs["organism_origin"], "organism_prediction.csv")
df_org_overall.to_csv(org_overall_csv, index=False)
print(f"Overall organism predictions saved to: {org_overall_csv}")

print("\n=== Functional & Organism Prediction Pipeline Completed ===")